## Enriching stock market data using Open AI API 

<p align="center">
    <img src="images/nasdaq100.png" width="450">
</p>

The Nasdaq-100 is a stock market index made up of 101 equity securities issued by 100 of the largest non-financial companies listed on the Nasdaq stock exchange. It helps investors compare stock prices with previous prices to determine market performance.

In this project you are provided with two CSV files containing Nasdaq-100 stock information:
- _**nasdaq100_CA.csv**_: contains information about companies in the index such as symbol, name, etc. For this analysis, only companies headquartered in California have been selected.
- _**nasdaq100_price_change.csv**_: contains price changes per stock across periods including (but not limited to) one day, five days, one month, six months, one year, etc.

As an AI developer, you will leverage the OpenAI API to classify companies into sectors and produce a summary of sector and company performance for this year, for the companies in the index that are headquartered in **California.**

# CSV with Nasdaq-100 stock data

In this project, you have available two CSV files `nasdaq100_CA.csv` and `nasdaq100_price_change.csv`.

## nasdaq100_CA.csv

```py
symbol,name,headQuarter,dateFirstAdded,cik,founded
AAPL,Apple Inc.,"Cupertino, CA",,0000320193,1976-04-01
ABNB,Airbnb,"San Francisco, CA",,0001559720,2008-08-01
ADBE,Adobe Inc.,"San Jose, CA",,0000796343,1982-12-01
...
```

## nasdaq100_price_change.csv

```py
symbol,1D,5D,1M,3M,6M,ytd,1Y,3Y,5Y,10Y,max
AAPL,-1.7254,-8.30086,-6.20411,3.042,15.64824,42.99992,8.47941,60.96299,245.42031,976.99441,139245.53954
ABNB,2.1617,-2.21919,9.88336,19.43286,19.64241,68.66902,23.64013,-1.04347,-1.04347,-1.04347,-1.04347
ADBE,0.5409,-1.77817,9.16191,52.0465,38.01522,57.22723,21.96206,17.83037,109.05718,1024.69214,251030.66399
ADI,0.9291,-4.03352,2.58486,3.65887,5.01602,17.02062,8.09735,63.42847,92.81874,286.77518,26012.63736
...
```

In [72]:
import os
import pandas as pd
from openai import OpenAI

client = OpenAI()

In [73]:
import pandas as pd
import datetime

nasdaq100_CA = pd.read_csv('nasdaq100_CA.csv')
nasdaq100_CA['founded'] = pd.to_datetime(nasdaq100_CA['founded'])
nasdaq100_CA['Ytd'] = nasdaq100_CA['founded'].dt.year
nasdaq100_CA.head()

,symbol,name,headQuarter,dateFirstAdded,cik,founded,Ytd
0,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01,1976
1,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01,2008
2,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01,1982
3,ADSK,Autodesk,"San Rafael, CA",NaN,769397,1982-01-30,1982
4,AMAT,Applied Materials,"Santa Clara, CA",NaN,6951,1967-11-10,1967


In [74]:
nasdaq100_price = pd.read_csv('nasdaq100_price_change.csv')
nasdaq100_price.head()

,symbol,1D,5D,1M,3M,6M,ytd,1Y,3Y,5Y,10Y,max
0,AAPL,-1.7254,-8.30086,-6.20411,3.04200,15.64824,42.99992,8.479410,60.96299,245.42031,976.99441,139245.53954
1,ABNB,2.1617,-2.21919,9.88336,19.43286,19.64241,68.66902,23.640130,-1.04347,-1.04347,-1.04347,-1.04347
2,ADBE,0.5409,-1.77817,9.16191,52.04650,38.01522,57.22723,21.962060,17.83037,109.05718,1024.69214,251030.66399
3,ADI,0.9291,-4.03352,2.58486,3.65887,5.01602,17.02062,8.097350,63.42847,92.81874,286.77518,26012.63736
4,ADP,2.0589,2.35462,14.66581,16.40059,10.60546,5.53732,0.888943,81.76679,81.87224,248.40950,27613.11042


In [75]:
nasdaq100_CA.shape

(38, 7)

In [76]:
sectors = ['Technology', 'Consumer Cyclical', 'Industrials', 'Utilities', 'Healthcare', 'Communication', 'Energy', 'Consumer Defensive', 'Real Estate', 'Financial']

sector_list = []

for index, company in nasdaq100_CA.iterrows():
    prompt = f"""
        Classyify company {company['symbol']} into one of the following sectors: {', '.join(sectors)}.
        Answer only with the sector name.
        For additional information about the company:
        Name: {company['name']}
        headQuarter: {company['headQuarter']}
    """

    response = client.chat.completions.create(
        model='gpt-3.5-turbo',
        messages=[
            {"role" : "system", "content" : "You are a financial advisor."},
            {"role" : "user", "content" : prompt}
        ],
        temperature=0
    )

    sector = response.choices[0].message.content.strip()
    sector_list.append(sector)

nasdaq100_CA['Sector'] = sector_list

In [77]:
nasdaq100_CA

,symbol,name,headQuarter,dateFirstAdded,cik,founded,Ytd,Sector
0,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01,1976,Technology
1,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01,2008,Real Estate
2,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01,1982,Technology
3,ADSK,Autodesk,"San Rafael, CA",NaN,769397,1982-01-30,1982,Technology
4,AMAT,Applied Materials,"Santa Clara, CA",NaN,6951,1967-11-10,1967,Technology
5,AMD,Advanced Micro Devices Inc.,"Santa Clara, CA",NaN,2488,1969-05-01,1969,Technology
6,AMGN,Amgen,"Thousand Oaks, CA",NaN,318154,1980-04-08,1980,Healthcare
7,ATVI,Activision Blizzard,"Santa Monica, CA",NaN,718877,2008-07-10,2008,Communication
8,AVGO,Broadcom Inc.,"San Jose, CA",NaN,1730168,1961-01-01,1961,Technology
9,CDNS,Cadence Design Systems,"San Jose, CA",NaN,813672,1988-01-01,1988,Technology


In [78]:
print(nasdaq100_CA['Sector'].value_counts())

Technology            26
Healthcare             5
Communication          2
Consumer Cyclical      2
Real Estate            1
Consumer Defensive     1
Financial              1
Name: Sector, dtype: int64


In [79]:
nasdaq100_price.shape

(101, 12)

In [80]:
nasdaq100_ca = nasdaq100_CA.merge(nasdaq100_price, on='symbol')
nasdaq100_ca.head()

,symbol,name,headQuarter,dateFirstAdded,cik,founded,Ytd,Sector,1D,5D,1M,3M,6M,ytd,1Y,3Y,5Y,10Y,max
0,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01,1976,Technology,-1.7254,-8.300860,-6.20411,3.04200,15.64824,42.99992,8.47941,60.96299,245.42031,976.99441,139245.53954
1,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01,2008,Real Estate,2.1617,-2.219190,9.88336,19.43286,19.64241,68.66902,23.64013,-1.04347,-1.04347,-1.04347,-1.04347
2,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01,1982,Technology,0.5409,-1.778170,9.16191,52.04650,38.01522,57.22723,21.96206,17.83037,109.05718,1024.69214,251030.66399
3,ADSK,Autodesk,"San Rafael, CA",NaN,769397,1982-01-30,1982,Technology,1.0641,-2.738120,1.06415,4.50139,-8.91348,10.02701,-7.69178,-13.76778,50.68792,483.87790,40643.00000
4,AMAT,Applied Materials,"Santa Clara, CA",NaN,6951,1967-11-10,1967,Technology,3.2404,-0.159341,7.74522,29.57091,24.77597,55.46366,39.46026,136.55812,203.00222,846.38137,157389.05599


In [81]:
prompt = f"""
    Provide summary information about Nasdaq-100 stock performance year to date (YTD) of companies headquartered in CA, recommending the two best sectors and three popular companies per sector.
    Company data: {nasdaq100_ca}
"""

response = client.chat.completions.create(
    model='gpt-3.5-turbo',
    messages=[{'role' : 'user', 'content' : prompt}],
    temperature=0
)

stock_recommendations = response.choices[0].message.content
print(stock_recommendations)

The Nasdaq-100 stock performance year to date (YTD) of companies headquartered in CA shows a mix of positive and negative returns. The two best sectors based on performance are Technology and Healthcare.

Three popular companies in the Technology sector are:
1. Apple Inc. (AAPL)
2. Adobe Inc. (ADBE)
3. Nvidia (NVDA)

Three popular companies in the Healthcare sector are:
1. Amgen (AMGN)
2. Gilead Sciences (GILD)
3. Illumina, Inc. (ILMN)

Overall, the stock performance of these companies varies, with some showing strong growth while others have experienced declines. Investors should carefully analyze each company's financials and market trends before making investment decisions.
